In [1]:
import pandas as pd
import string
import operator
from pprint import pprint
import tqdm

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
import gensim
from gensim.utils import simple_preprocess
from gensim import corpora, models
from gensim.models import CoherenceModel
import pyLDAvis.gensim
from gensim.models import CoherenceModel

In [4]:
import nltk
from stop_words import get_stop_words
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
import re
porter_stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer() 

/opt/anaconda3/lib/python3.7/site-packages/nltk/decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
/opt/anaconda3/lib/python3.7/site-packages/nltk/lm/counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence, defaultdict
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jackymeister/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jackymeister/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics

In [6]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

from keras.preprocessing import text, sequence
from keras import layers, optimizers

Using TensorFlow backend.


In [7]:
import matplotlib.pyplot as plt

In [8]:
stop_words = list(get_stop_words('en'))         #About 900 stopwords
#print(stop_words)
nltk_words = list(stopwords.words('english')) #About 150 stopwords
#print(nltk_words)
stop_words.extend(nltk_words)
#print(stop_words)

In [9]:
Twitter = pd.read_csv("Google.csv")
Twitter.head()

,Unnamed: 0,Text,Timestamp,RelatedActiveTopics,PlatformType,Platform,Year
0,1,"The business, function and art of textiles | T...",2020-01-15 23:32:18,digital cloth,NewsAggregator,GoogleNews,2020
1,2,"Milly &amp; Roots, The Headscarf. A children's...",2020-01-10 18:00:00,digital cloth,NewsAggregator,GoogleNews,2020
2,3,Pamex 2020: Epson - A market leader in textile...,2020-01-07 18:00:00,digital cloth,NewsAggregator,GoogleNews,2020
3,4,Pamplin Media Group - K's Clothing Boutique a ...,2020-01-13 18:00:00,digital cloth,NewsAggregator,GoogleNews,2020
4,5,From selling shirts to owning ‘Tuned in Tokyo’...,2020-01-02 18:00:00,digital cloth,NewsAggregator,GoogleNews,2020


In [10]:
#Twitter = Twitter.sample(n = 3000)
Twitter = Twitter.reset_index(drop = True)
Twitter.head()

,Unnamed: 0,Text,Timestamp,RelatedActiveTopics,PlatformType,Platform,Year
0,1,"The business, function and art of textiles | T...",2020-01-15 23:32:18,digital cloth,NewsAggregator,GoogleNews,2020
1,2,"Milly &amp; Roots, The Headscarf. A children's...",2020-01-10 18:00:00,digital cloth,NewsAggregator,GoogleNews,2020
2,3,Pamex 2020: Epson - A market leader in textile...,2020-01-07 18:00:00,digital cloth,NewsAggregator,GoogleNews,2020
3,4,Pamplin Media Group - K's Clothing Boutique a ...,2020-01-13 18:00:00,digital cloth,NewsAggregator,GoogleNews,2020
4,5,From selling shirts to owning ‘Tuned in Tokyo’...,2020-01-02 18:00:00,digital cloth,NewsAggregator,GoogleNews,2020


In [11]:
len(Twitter)

2624

In [12]:
Twitter = Twitter.drop_duplicates("Text")
len(Twitter)

2624

In [13]:
T_Text = Twitter["Text"]

In [14]:
T_Text["Index"] = Twitter.set_index
T_Text = pd.DataFrame(T_Text)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
T_Text.head()

,Text
0,"The business, function and art of textiles | T..."
1,"Milly &amp; Roots, The Headscarf. A children's..."
2,Pamex 2020: Epson - A market leader in textile...
3,Pamplin Media Group - K's Clothing Boutique a ...
4,From selling shirts to owning ‘Tuned in Tokyo’...


In [16]:
T_Text = T_Text[T_Text['Text'].notnull()]
T_Text.loc[:,"Text"] = T_Text.Text.apply(lambda x : " ".join(re.findall('\w+',str(x)))) # remove punctuation
T_Text.loc[:,"Text"] = T_Text.Text.apply(lambda x : str.lower(x)) # lower case
T_Text["Text"] = T_Text.Text.str.replace('\d+', '') # remove numbers

<>:2: DeprecationWarning: invalid escape sequence \w
<>:4: DeprecationWarning: invalid escape sequence \d
<>:2: DeprecationWarning: invalid escape sequence \w
<>:4: DeprecationWarning: invalid escape sequence \d
<>:2: DeprecationWarning: invalid escape sequence \w
<>:4: DeprecationWarning: invalid escape sequence \d
<ipython-input-16-e26ed5b07c42>:2: DeprecationWarning: invalid escape sequence \w
  T_Text.loc[:,"Text"] = T_Text.Text.apply(lambda x : " ".join(re.findall('\w+',str(x)))) # remove punctuation
<ipython-input-16-e26ed5b07c42>:4: DeprecationWarning: invalid escape sequence \d
  T_Text["Text"] = T_Text.Text.str.replace('\d+', '') # remove numbers


In [17]:
def remove_stopWords(s):
    s = ' '.join(word for word in s.split() if word not in stop_words)
    return s
T_Text.loc[:,"Text"] = T_Text.Text.apply(lambda x: remove_stopWords(x)) # stop words

In [18]:
def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

T_Text.loc[:,"Text"] = T_Text.Text.apply(stem_sentences)

In [19]:
def lemm_sentences(sentence):
    tokens = sentence.split()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens

T_Text.loc[:,"Text"] = T_Text.Text.apply(lemm_sentences)

In [20]:
T_Text.head()

,Text
0,"[busi, function, art, textil, time, georgian, ..."
1,"[milli, amp, root, headscarf, child, book, dea..."
2,"[pamex, epson, market, leader, textil, print, ..."
3,"[pamplin, medium, group, k, cloth, boutiqu, gr..."
4,"[sell, shirt, own, tune, tokyo, franki, quiroz..."


In [21]:
dictionary = gensim.corpora.Dictionary(T_Text["Text"])

In [22]:
count = 0
for k, v in dictionary.items():
    print(k, v)
    count += 1
    if count > 10:
        break

0 art
1 busi
2 function
3 georgian
4 textil
5 time
6 amp
7 book
8 child
9 deal
10 digit


In [23]:
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=1000000)

In [24]:
# doc2bow
bow_corpus = [dictionary.doc2bow(doc) for doc in T_Text["Text"]]

# LDA using BOW Base Model
lda_model_base = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)
coherence_model_lda_base = CoherenceModel(model=lda_model_base, texts = T_Text["Text"], dictionary = dictionary, coherence='c_v')
coherence_lda_base = coherence_model_lda_base.get_coherence()
print('\nCoherence Score: ', coherence_lda_base)

In [ ]:
lda_model = gensim.models.LdaMulticore(corpus=bow_corpus,
                                           id2word=dictionary,
                                           num_topics=13, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.9,
                                           eta=0.1)
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts = T_Text["Text"], dictionary = dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
lda_display = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)


In [25]:
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=bow_corpus,
                                           id2word=dictionary,
                                           num_topics=10, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b,
                                           per_word_topics=True)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=T_Text["Text"], dictionary=dictionary, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [26]:
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 5
max_topics = 16
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(bow_corpus)
corpus_sets = [#gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               #gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               #gensim.utils.ClippedCorpus(bow_corpus, num_of_docs*0.75), 
               bow_corpus]
corpus_title = ['100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total = 50)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=bow_corpus[i], dictionary=dictionary, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results_Google.csv', index=False)
    pbar.close()

330it [2:05:32, 22.82s/it]                     


In general, higher alpha values mean documents contain more similar topic contents. The same is true for beta, but with topics and words: generally a high beta will result in topics with more similar word contents. Also, an asymmetric alpha is helpful, where as an asymmetric beta is largely not.

First we decide num_topics by controling alpha and beta.
Then we get our number of topics, then we decide alpha and beta to get our model. 

In [27]:
lda_model = gensim.models.LdaMulticore(corpus=bow_corpus,
                                           id2word=dictionary,
                                           num_topics=8, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.01,
                                           eta=0.01)

In [28]:
for idx, topic in lda_model.print_topics(-1, num_words=50):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.021*"realiti" + 0.011*"virtual" + 0.010*"augment" + 0.008*"ar" + 0.007*"vr" + 0.007*"digit" + 0.006*"said" + 0.005*"game" + 0.004*"world" + 0.004*"experi" + 0.004*"work" + 0.004*"one" + 0.004*"develop" + 0.003*"mix" + 0.003*"glass" + 0.003*"busi" + 0.003*"real" + 0.003*"see" + 0.003*"eye" + 0.003*"prosthesi" + 0.003*"headset" + 0.003*"design" + 0.003*"say" + 0.003*"peopl" + 0.003*"need" + 0.003*"creat" + 0.003*"first" + 0.002*"way" + 0.002*"ago" + 0.002*"prosthet" + 0.002*"user" + 0.002*"vision" + 0.002*"content" + 0.002*"look" + 0.002*"industri" + 0.002*"news" + 0.002*"team" + 0.002*"custom" + 0.002*"go" + 0.002*"servic" + 0.002*"tech" + 0.002*"train" + 0.002*"face" + 0.002*"right" + 0.002*"app" + 0.002*"brand" + 0.002*"healthcar" + 0.002*"hour" + 0.002*"offer" + 0.002*"take"
Topic: 1 
Words: 0.007*"one" + 0.005*"said" + 0.004*"smart" + 0.004*"look" + 0.004*"say" + 0.004*"go" + 0.003*"show" + 0.003*"want" + 0.003*"peopl" + 0.003*"come" + 0.003*"first" + 0.003*"clean

In [29]:
def format_topics_sentences(ldamodel=lda_model, corpus=bow_corpus, texts=T_Text["Text"]):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[bow_corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Percentage Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=bow_corpus, texts=T_Text["Text"])


df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']



In [ ]:
df_dominant_topic["Text"] = df_dominant_topic["Text"].str.replace(',','')

In [30]:
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,4.0,0.9913,"robot, research, system, digit, develop, desig...","[busi, function, art, textil, time, georgian, ..."
1,1,4.0,0.6575,"robot, research, system, digit, develop, desig...","[milli, amp, root, headscarf, child, book, dea..."
2,2,4.0,0.9049,"robot, research, system, digit, develop, desig...","[pamex, epson, market, leader, textil, print, ..."
3,3,1.0,0.8880,"one, said, smart, look, say, go, show, want, p...","[pamplin, medium, group, k, cloth, boutiqu, gr..."
4,4,1.0,0.3653,"one, said, smart, look, say, go, show, want, p...","[sell, shirt, own, tune, tokyo, franki, quiroz..."
5,5,6.0,0.8337,"global, analysi, research, industri, region, g...","[global, optic, brighten, market, reach, u, bn..."
6,6,1.0,0.9135,"one, said, smart, look, say, go, show, want, p...","[christ, bodyguard, men, protect, philippin, r..."
7,7,1.0,0.7434,"one, said, smart, look, say, go, show, want, p...","[th, chitra, santh, varieti, buyer, found, dea..."
8,8,4.0,0.6862,"robot, research, system, digit, develop, desig...","[heimtextil, launch, innov, new, trend, interi..."
9,9,5.0,0.3524,"devic, wearabl, medic, smart, health, monitor,...","[fit, tracker, use, explod, u, especi, among, ..."


In [31]:
df_dominant_topic.to_csv("LDA Result_Google.csv")